In [1]:
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import unicodedata as ud
from sklearn.metrics import precision_recall_fscore_support as prfs
import csv
import HTMLParser
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.cross_validation import cross_val_score
from numpy import random
#from stemming.porter2 import stem
from nltk.stem.porter import PorterStemmer as porterStemmer
from itertools import izip
from sklearn.ensemble import RandomForestClassifier

In [2]:
def cleanTweet(originalTweet):
    htmlParser = HTMLParser.HTMLParser()

    tweet = originalTweet
    #tweet = htmlParser.unescape(originalTweet);
    #tweet = tweet.decode('windows-1252').encode('ascii', 'ignore')
    #tweet = tweet.decode('windows-1252')
    #tweet = tweet.decode("utf8").encode('ascii', 'ignore')
    #tweet = re.sub(r'[^\x00-\xFF]+', r'', tweet)
    #tweet = re.sub(r'[^\x00-\x7F]+', r'', tweet)
    #tweet = tweet.decode('utf-8').strip()
    #tweet = tweet.decode('unicode_escape').encode('ascii','ignore')
    #tweet = tweet.encode('ascii','ignore')
    tweet = ''.join([i if ord(i) < 128 else ' ' for i in tweet])
    
    # remove URLs in tweet
    tweet = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', tweet)

    # remove strings starting with @ in tweet
    tweet = re.sub(r'(\s)@\w+', r'', tweet)
    tweet = re.sub(r'@\w+', r'', tweet)
    
    # remove HTML tags from tweet
    tweet = re.sub('<[^<]+?>', '', tweet)

    # separates words joined with capital words.
    # E.g. DisplayIsAweson to Display Is Awesom
    #tweet = " ".join(re.findall('[A-Z][^A-Z]*', tweet));

    # remove exclamations
    tweet = re.sub(r'[<>!#@$:.,%\?-]+', r'', tweet)

    # remove extra white spaces
    tweet = re.sub(r'\s+', r' ', tweet)
    
    # stemming
    stemmer = porterStemmer()
    stemmedTweet = [stemmer.stem(word) for word in tweet.split(" ")]
    stemmedTweet = " ".join(stemmedTweet)
    
    tweet = str(stemmedTweet)
    tweet = tweet.replace("'", "")
    
    return tweet

In [3]:
#data = pd.read_csv("A:\\new_Sync\\Box Sync\\academics\\sem3\\583\\project\\TwitterAnalysis\\data\\trainRomney_3_columns.csv")

In [4]:
data = pd.read_csv("A:\\new_Sync\\Box Sync\\academics\\sem3\\583\\project\\TwitterAnalysis\\data\\trainObama_3_columns.csv")

In [5]:
len(data)

5624

In [6]:
rawTweetsSeries = data['Anootated tweet'];
tweetLabels = data['Class'];

In [7]:
rawTweetsList = rawTweetsSeries.tolist()
tweetLabelList = tweetLabels.tolist()

In [8]:
len(rawTweetsList)

5624

In [9]:
randomTweets = random.choice(rawTweetsList, 5)

In [10]:
#randomTweets

In [11]:
# clean all the tweets
i = 0;
cleanedTweetsList = []
for tweet in rawTweetsList:
    #tweet.encode('utf-8').strip()
    #tweet = tweet.decode("utf8").encode('ascii', 'ignore')
    #print (i ,),
    cleanedTweet = cleanTweet(tweet).encode('ascii', 'ignore').strip();
    cleanedTweetsList.append(cleanedTweet);
    
    i += 1

In [12]:
#print (i)
#print (len(cleanedTweetsList))
#print (random.choice(cleanedTweetsList, 5))
#print (type(cleanedTweetsList[0]))

In [13]:
#with open('cleanedRomney.csv', 'wb') as f:
#    writer = csv.writer(f)
#    writer.writerows(izip(cleanedTweetsList, tweetLabelList))

In [14]:
#documents = [[stem(word) for word in sentence.split(" ")] for sentence in documents]

In [15]:
trainTweets, testTweets, trainTweetLabels, testTweetLabels = train_test_split(cleanedTweetsList, tweetLabelList, 
                                                                              test_size=0.15, random_state = 40);

In [16]:
# Create feature vectors
vectorizer = TfidfVectorizer(min_df=0.00125,
                             max_df = 0.70,
                             sublinear_tf=True,
                             use_idf=True,
                            stop_words=u'english',
                            analyzer='word',
                            ngram_range=(1,6),lowercase=True)

In [17]:
train_vectors = vectorizer.fit_transform(trainTweets);
test_vectors = vectorizer.transform(testTweets);
total_vectors = vectorizer.fit_transform(cleanedTweetsList);

In [18]:
classifier = RandomForestClassifier(n_estimators=18, criterion="entropy")

In [19]:
classifier.fit(train_vectors, trainTweetLabels)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=18, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [20]:
prediction = classifier.predict(test_vectors)

In [21]:
predictedLabelList = prediction.tolist()

In [22]:
classActual = np.array(testTweetLabels)

classPredicted = np.array(predictedLabelList)

In [23]:
target_names = ['-1', '0', '1']

In [24]:
prfs(classActual, classPredicted)

(array([ 0.5795053,  0.503125 ,  0.6846473]),
 array([ 0.5754386 ,  0.56293706,  0.6043956 ]),
 array([ 0.57746479,  0.53135314,  0.64202335]),
 array([285, 286, 273], dtype=int64))

In [25]:
print (len(predictedLabelList))
print (len(testTweetLabels))

844
844


In [26]:
correct = 0;
for i in range(len(predictedLabelList)):
    if(predictedLabelList[i] == testTweetLabels[i]):
        correct += 1;

In [27]:
predictedLabelList[170]

1L

In [28]:
import collections
counter=collections.Counter(predictedLabelList)
print(counter)

Counter({0L: 320, -1L: 283, 1L: 241})


In [29]:
import collections
counter=collections.Counter(testTweetLabels)
print(counter)

Counter({0: 286, -1: 285, 1: 273})


In [30]:
classifier.score(test_vectors, testTweetLabels)

0.58056872037914697

In [31]:
print(classification_report(testTweetLabels, predictedLabelList, target_names=target_names))

             precision    recall  f1-score   support

         -1       0.58      0.58      0.58       285
          0       0.50      0.56      0.53       286
          1       0.68      0.60      0.64       273

avg / total       0.59      0.58      0.58       844



In [32]:
cvScores = cross_val_score(classifier, total_vectors, tweetLabelList, cv=10)

In [33]:
print ("Mean: ", cvScores.mean())
print ("Minimum: ", cvScores.min())
print ("Maximum: ", cvScores.max())

('Mean: ', 0.54888666488711491)
('Minimum: ', 0.49555950266429838)
('Maximum: ', 0.58969804618117228)


In [34]:
# stemming tweets
stemmer = porterStemmer()
stemmedTweetsList = [[stemmer.stem(word) for word in sentence.split(" ")] for sentence in cleanedTweetsList]

In [35]:
print(type(stemmedTweetsList[0]))
mystr = str(stemmedTweetsList[0])
print (type(mystr))

<type 'list'>
<type 'str'>


In [36]:
#stemmedTweetsList = [" ".join(sentence) for sentence in stemmedTweetsList]

In [37]:
#stemmedTweetsList